# Avance 2: Descarga de datos de negocios desde la API de yelp:

Se importan los modulos a utilizar. La API de yelp es de tipo REST, por lo que se importa requests para realizar y recibir payloads en http.
Adicionalmente, se importa load_dotenv poder acceder a las variables de environment que tengamos y os para poder cargar esas variables de enviroment. Esto es con el fin de manejar de manera segura las credenciales de aceso a la API Yelp (Las cuales estan en un archivo .env).

En el archivo .env.example se tiene la estructura de como deben ir registrados los datos de usario y contraseña. **Se deben ingresar en este archivo las credenciales adecuadas de aceso y ademas cambiar el nombre de archivo de .env.example a .env**

In [47]:
import requests
import pandas as pd
import os
import load_dotenv
import numpy as np

In [30]:
# Obtener las credenciales de la API desde las variables de entorno. Deberías tener un archivo .env con estas variables definidas.
load_dotenv.load_dotenv()
cliente_id = os.getenv("CLIENTE_ID")
api_key = os.getenv("API_KEY")
headers = {'Authorization':'Bearer %s'%api_key}

Ya teniendo la api_key y el cliente_id, se aplicaran los http requests al endpoint de la API: https://api.yelp.com/v3/businesses/search

De acuerdo a la documentacion de Yelp, la variable 'term' puede contener categorias y 'location' puede usarse para filtrar por ciudad. En este caso usaremos 'restaurants' y 'Chicago' como se ve a continuacion:

In [31]:
# Parámetros de búsqueda (puedes ajustarlos según tus necesidades). Se define 
url = 'https://api.yelp.com/v3/businesses/search'
ciudad = 'Chicago'
term='restaurants'

Adicionalmente, la API solo acepta 50 filas por request y hasta un maximo de 240 filas. Por ende, para recolectar todos los posibles datos se crea un loop en el que se adicionan todas las consultas a una lista:

In [32]:
# se define un limite de 30 filas por request y un maximo de 240 filas
limit=30
max_entries= 240

# Realizar múltiples solicitudes para obtener más resultados de acuerdo con el límite y el offset especificados anteriormente
resultados=[]
for offset in range(0, max_entries, limit):
    params = {'term':term, 'location':ciudad, 'limit':limit, 'offset':offset}
    response=requests.get(url, headers=headers, params=params)
    if response.status_code != 200:
        print("Error:", response.json())
        break
    
    data = response.json()
    negocios = data.get("businesses", []) # obtener la lista de negocios de la respuesta, si no existe, asigna una lista vacía

    if not negocios:  # si ya no hay más resultados
        break

    resultados.extend(negocios)


La lista de dictionarios se convierte ahora a dataframe:

In [33]:
# Convertir la lista de resultados en un DataFrame de pandas
df_negocios = pd.json_normalize(resultados)

In [34]:
# Se busca entender la estructura del dataframe
df_negocios.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 240 entries, 0 to 239
Data columns (total 24 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   id                        240 non-null    object 
 1   alias                     240 non-null    object 
 2   name                      240 non-null    object 
 3   image_url                 240 non-null    object 
 4   is_closed                 240 non-null    bool   
 5   url                       240 non-null    object 
 6   review_count              240 non-null    int64  
 7   categories                240 non-null    object 
 8   rating                    240 non-null    float64
 9   transactions              240 non-null    object 
 10  price                     158 non-null    object 
 11  phone                     240 non-null    object 
 12  display_phone             240 non-null    object 
 13  distance                  240 non-null    float64
 14  coordinate

La variable costo tiene bastantes filas faltantes. Se calculara ahora el porcentaje:

In [35]:
print('Porcentaje de valores faltantes en costo: ',df_negocios['price'].isna().mean() * 100)

Porcentaje de valores faltantes en costo:  34.166666666666664


La moda es 'medio' por lo cual se puede utilizar para rellenar los datos faltantes, ya que es un valor representativo de la muestra y seria util para un analisis descriptivo. Sin embargo, al haber un porcentaje relativamente alto de datos faltantes, puede que esto sesgue los resultados hacia la moda. 

La otra alternativa es reemplazar los NaN asignando valores aleatorios con la misma probabilidad de la distribución observada: es decir, un 58% sea medio, un 32% alto, un 10% muy alto y el resto bajo. No obstante, esto introduciria aleatoriedad y los resultados podrian variar en cada ejecucion.

No obstante, como se observa abajo, se decidio reemplazar los valores nulos por desconocido. Esto es porque al final se quiere dar recomendaciones a las empresas sobre que informacion es util que tengan en su base de datos de yelp o que promocionen, por lo que por ejemplo se podria comparar el desempeño de restaurantes con valores desconocidos de variables como categoria de precio para verificar el impacto de tenerlo o no.

Tambien, para efectos del estudio, se convertiran los simbolos a $, $$, $$$ y $$$$ a bajo, medio, alto y muy alto, como se ve a continuacion:

In [ ]:
df_negocios['price']=df_negocios['price'].replace({'$':'bajo','$$':'medio','$$$':'alto','$$$$':'muy alto'})

Finalmente, las direcciones no son relevantes en este estudio, asi que se borraran, con excepcion del zipcode, ya que se podra usar mas adelante para ver la concentracion de los negocios por zona:

In [44]:
df_negocios.drop(columns=['coordinates.latitude','coordinates.longitude','location.address1', 'location.address2', 'location.address3', 'location.country', 'location.state', 'location.display_address','location.city','distance','display_phone'], inplace=True)

Ahora el dataset se visualiza un poco para observar las variables y su formato:

In [25]:
df_negocios.head()

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,transactions,price,phone,location.zip_code
0,qjnpkS8yZO8xcyEIy5OU9A,girl-and-the-goat-chicago,Girl & The Goat,https://s3-media0.fl.yelpcdn.com/bphoto/ya6gjD...,False,https://www.yelp.com/biz/girl-and-the-goat-chi...,10510,"[{'alias': 'newamerican', 'title': 'New Americ...",4.4,[delivery],alto,+13124926262,60607
1,boE4Ahsssqic7o5wQLI04w,the-purple-pig-chicago,The Purple Pig,https://s3-media0.fl.yelpcdn.com/bphoto/rHHvhR...,False,https://www.yelp.com/biz/the-purple-pig-chicag...,8856,"[{'alias': 'tapasmallplates', 'title': 'Tapas/...",4.3,"[delivery, pickup]",alto,+13124641744,60611
2,riT822EnU7y_5eCuJsd9sA,cindys-rooftop-chicago,Cindy's Rooftop,https://s3-media0.fl.yelpcdn.com/bphoto/XLdHyZ...,False,https://www.yelp.com/biz/cindys-rooftop-chicag...,2717,"[{'alias': 'newamerican', 'title': 'New Americ...",4.1,[delivery],medio,+13127923502,60603
3,gzhkdb6YoiFm5s3vriG1AA,gretel-chicago,Gretel,https://s3-media0.fl.yelpcdn.com/bphoto/-dxigk...,False,https://www.yelp.com/biz/gretel-chicago?adjust...,415,"[{'alias': 'cocktailbars', 'title': 'Cocktail ...",4.5,"[delivery, pickup]",medio,+17737703427,60647
4,VPJk-SEWSWS_nGoQvM-COw,penumbra-chicago,Penumbra,https://s3-media0.fl.yelpcdn.com/bphoto/IlSPQK...,False,https://www.yelp.com/biz/penumbra-chicago?adju...,996,"[{'alias': 'wine_bars', 'title': 'Wine Bars'},...",4.8,"[restaurant_reservation, delivery]",alto,+17737722343,60647


Observamos ademas, que el dataframe tiene listas de diccionarios en la columna categoria
Por ende, se expandira la lista de diccionarios para poder agrupar, agregar y visualizar por categoria y ver su impacto:

In [45]:
df_negocios['categories_list'] = df_negocios['categories'].apply( lambda x: [d['title'] for d in x] if isinstance(x, list) else [])

In [46]:
df_negocios=df_negocios.explode('categories_list').reset_index(drop=True)

Ahora, para inputar los datos faltantes de precio, se usara la moda de acuerdo a cada categoria, ya que la categoria es la que mayor influencia tiene en el precio de un restaurante;

In [48]:
df_negocios['price'] = df_negocios.groupby('categories_list')['price'].transform(lambda x: x.fillna(x.mode()[0] if not x.mode().empty else np.nan))

C:\Users\Carlos\AppData\Local\Temp\ipykernel_16524\3123470926.py:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_negocios['price'] = df_negocios.groupby('categories_list')['price'].transform(lambda x: x.fillna(x.mode()[0] if not x.mode().empty else np.nan))


El codigo de arriba deja como nan los valores de 'price' cuando todos los elemento dentro de esa categoria estan vacios. Por lo tanto, vamos a revisar si queda algun valor nulo en general

In [51]:
df_negocios['price'].isnull().sum()


22

Se realizara una imputacion a estos nulos de acuerdo con la moda en general para todo el dataset:

In [52]:
df_negocios['price']=df_negocios['price'].fillna(df_negocios['price'].mode()[0])

Finalmente, se exportan los datos a csv para poderlos trabajar mas a fondo en el avance 3

In [53]:
df_negocios.to_csv("df_negocios_chicago.csv", index=False)

El resto del proyecto continuara en el avance 3. Hasta este punto, se ha logrado realizar una limpieza total de la base de datos de Yelp para restaurantes